<a href="https://colab.research.google.com/github/thinpine/pure-tv/blob/main/datascrapingfinal_2024_08_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trials with fake user agent (failed)**

In [ ]:
pip install fake-useragent

In [ ]:
# Import the required libraries

import requests

from fake_useragent import UserAgent



# Instantiate the UserAgent class

ua = UserAgent()



# Get random user agents

random_ua = ua.random



# Specify the request URL

url = 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=2019-01-01,2024-07-01&genres=!documentary,!short&countries=TR&sort=release_date,asc'



# Pass the random user agents to the user-agent headers

request_headers = {

    'user-agent': random_ua

}



# Make a get request to the URL and get the response

response = requests.get(url, headers= request_headers)



# Resolve response and print the user agent information

if response.status_code == 200:

    print(response.request.headers['User-Agent'])

else:

    print(response.status_code)


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0


# **Extracts IMDB ID's from IMDB html text**

In [ ]:
def extract_titles(file_path):
    titles = []  # To store the extracted titles

    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    start = 0  # Position to start searching for the next 'title/tt'
    while True:
        # Find the next occurrence of 'title/tt'
        start = data.find('title/tt', start)
        if start == -1:  # If 'title/tt' is not found, stop the loop
            break

        # Move the position to after 'title/tt'
        start += len('title/tt')

        # Find the position of the next '/'
        end = data.find('/', start)

        # Extract the string from 'tt' to the next '/'
        if end != -1:
            titles.append('tt' + data[start:end])

    return titles

# Example usage
file_path = '/content/fullhtmltext_filtered.txt'
titles_array = extract_titles(file_path)
print(titles_array) #Array contains two instances of every IMDB ID, each being back to back.
print(len(titles_array)/2) #Number of shows


['tt9520396', 'tt9520396', 'tt11282156', 'tt11282156', 'tt11637968', 'tt11637968', 'tt9471962', 'tt9471962', 'tt14864648', 'tt14864648', 'tt9529618', 'tt9529618', 'tt9529892', 'tt9529892', 'tt9525880', 'tt9525880', 'tt9355244', 'tt9355244', 'tt9529500', 'tt9529500', 'tt9826628', 'tt9826628', 'tt16711822', 'tt16711822', 'tt19849996', 'tt19849996', 'tt9667744', 'tt9667744', 'tt25148102', 'tt25148102', 'tt9540596', 'tt9540596', 'tt9808672', 'tt9808672', 'tt9721776', 'tt9721776', 'tt9732208', 'tt9732208', 'tt9910728', 'tt9910728', 'tt9915022', 'tt9915022', 'tt10137654', 'tt10137654', 'tt10098792', 'tt10098792', 'tt9477680', 'tt9477680', 'tt10147676', 'tt10147676', 'tt10278814', 'tt10278814', 'tt10105930', 'tt10105930', 'tt31630338', 'tt31630338', 'tt10319992', 'tt10319992', 'tt10376348', 'tt10376348', 'tt14055540', 'tt14055540', 'tt10377430', 'tt10377430', 'tt10324306', 'tt10324306', 'tt10374050', 'tt10374050', 'tt10319866', 'tt10319866', 'tt8846682', 'tt8846682', 'tt10611748', 'tt10611748

# **Code I ended up with**

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
def extract_titles(file_path):
    titles = []  # To store the extracted titles

    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    start = 0  # Position to start searching for the next 'title/tt'
    while True:
        # Find the next occurrence of 'title/tt'
        start = data.find('title/tt', start)
        if start == -1:  # If 'title/tt' is not found, stop the loop
            break

        # Move the position to after 'title/tt'
        start += len('title/tt')

        # Find the position of the next '/'
        end = data.find('/', start)

        # Extract the string from 'tt' to the next '/'
        if end != -1:
            titles.append('tt' + data[start:end])

    return titles

# Example usage
file_path = '/content/fullhtmltext_filtered.txt'
imdb_ids = extract_titles(file_path)


def scrape_imdb_data(imdb_ids):
    # List to store the scraped data
    scraped_data = []

    # Base URL for IMDb
    base_url = "https://www.imdb.com/title/"

    for i in range(0, len(imdb_ids), 2):
        # Create the URL for the IMDb page
        url = base_url + imdb_ids[i]

        # Send an HTTP GET request to fetch the HTML content of the page
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve {imdb_ids[i]}")
            continue

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract relevant data (example fields)
        try:
            # Get the title of the movie/series
            title = soup.find('h1').text.strip()

            # Get the start and end dates (if available)
            date_range = soup.find('span', {'id': 'titleYear'}).text.strip() if soup.find('span', {'id': 'titleYear'}) else "N/A"

            # Get the number of episodes (if it's a series)
            episodes = soup.find('span', text='Episodes:').next_sibling.strip() if soup.find('span', text='Episodes:') else "N/A"

            # Get the parental certificate or guide
            certificate = soup.find('span', {'class': 'certificate'}).text.strip() if soup.find('span', {'class': 'certificate'}) else "N/A"

            # Get the stars (actors) listed
            stars_section = soup.find_all('a', href=lambda href: href and "/name/" in href)
            stars = ', '.join([star.text.strip() for star in stars_section[:5]])  # Limiting to the first 5 stars

            # Add the scraped data to the list
            scraped_data.append([imdb_ids[i], title, date_range, episodes, certificate, stars])

        except Exception as e:
            print(f"Error scraping {imdb_ids[i]}: {e}")

    return scraped_data

def write_to_csv(scraped_data, output_file):
    # Define the headers for the CSV file
    headers = ["IMDB ID", "Title", "Date Range", "Episode Count", "Certificate", "Stars"]

    # Write the data to a CSV file
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(scraped_data)

# Example usage
#imdb_ids = ["tt0111161", "tt0133093"]  # Replace with your list of IMDb IDs
scraped_data = scrape_imdb_data(imdb_ids)
write_to_csv(scraped_data, 'imdb_scraped_data.csv')




Failed to retrieve tt9520396
Failed to retrieve tt11282156
Failed to retrieve tt11637968
Failed to retrieve tt9471962
Failed to retrieve tt14864648
Failed to retrieve tt9529618
Failed to retrieve tt9529892
Failed to retrieve tt9525880
Failed to retrieve tt9355244
Failed to retrieve tt9529500
Failed to retrieve tt9826628
Failed to retrieve tt16711822
Failed to retrieve tt19849996
Failed to retrieve tt9667744
Failed to retrieve tt25148102
Failed to retrieve tt9540596
Failed to retrieve tt9808672
Failed to retrieve tt9721776
Failed to retrieve tt9732208
Failed to retrieve tt9910728
Failed to retrieve tt9915022
Failed to retrieve tt10137654
Failed to retrieve tt10098792
Failed to retrieve tt9477680
Failed to retrieve tt10147676
Failed to retrieve tt10278814
Failed to retrieve tt10105930
Failed to retrieve tt31630338
Failed to retrieve tt10319992
Failed to retrieve tt10376348
Failed to retrieve tt14055540
Failed to retrieve tt10377430
Failed to retrieve tt10324306
Failed to retrieve tt10374

KeyboardInterrupt: 

# **The code that can bypass 'Forbidden 403'**

In [ ]:
import requests
from bs4 import BeautifulSoup
import random


# crawl IMDB Top 250 and randomly select a movie
URL = 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=2019-01-01,2024-07-01&genres=!documentary,!short,!game-show,!reality-tv,!talk-show&countries=TR&sort=release_date,asc'
#response = requests.get(URL,  headers={"User-Agent": "Chrome"})
response = requests.get(URL,  headers={"User-Agent": "Mozilla/5.0"})

with open("hi.html", "w") as f:
    f.write(response.text)

soup = BeautifulSoup(response.text, 'html.parser')
links = [i for i in soup.find_all("a", {"class":"ipc-title-link-wrapper"}) if '/title/' in i['href']]
for i in range(25):
  selected_movie = links[i]
  print('http://www.imdb.com' + selected_movie['href'])
  print(selected_movie.text)

# **The combined + enhanced code**

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def extract_titles(file_path):
    titles = []  # To store the extracted titles

    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

    start = 0  # Position to start searching for the next 'title/tt'
    while True:
        # Find the next occurrence of 'title/tt'
        start = data.find('title/tt', start)
        if start == -1:  # If 'title/tt' is not found, stop the loop
            break

        # Move the position to after 'title/tt'
        start += len('title/tt')

        # Find the position of the next '/'
        end = data.find('/', start)

        # Extract the string from 'tt' to the next '/'
        if end != -1:
            titles.append('tt' + data[start:end])

    return titles

# Example usage
file_path = '/content/fullhtmltext_filtered.txt'
imdb_ids = extract_titles(file_path)


#Function to retrieve Parents Guide information

"""
def get_parents_guide_violence(imdb_id, headers):
    base_url = f"https://www.imdb.com/title/{imdb_id}/"
    response = requests.get(base_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve {imdb_id}")
        return "N/A"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the link to the Parents Guide page
    parents_guide_link = soup.find('a', href=lambda href: href and '/parentalguide' in href)
    if not parents_guide_link:
        return "N/A"

    # Extract the full URL for the Parents Guide page
    parents_guide_url = "https://www.imdb.com" + parents_guide_link['href']

    # Now send a request to the Parents Guide page
    guide_response = requests.get(parents_guide_url, headers=headers)
    if guide_response.status_code != 200:
        print(f"Failed to retrieve Parents Guide for {imdb_id}")
        return "N/A"

    guide_soup = BeautifulSoup(guide_response.text, 'html.parser')

    # Extract the 'Violence & Gore' rating
    violence_section = guide_soup.find('span', text='Violence & Gore')
    if violence_section:
        violence_info = violence_section.find_next('span', {'class': 'ipl-status-pill'}).text.strip()
        return violence_info
    else:
        return "N/A"
"""

def get_parents_guide_violence(imdb_id, headers):
    base_url = f"https://www.imdb.com/title/{imdb_id}/parentalguide"
    response = requests.get(base_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve Parents Guide for {imdb_id}")
        return "N/A"

    guide_soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the 'Violence & Gore' section by its ID
    violence_section = guide_soup.find('section', {'id': 'advisory-violence'})

    if not violence_section:
        return "N/A"

    # Look for the severity rating within the section (e.g., Mild, Moderate)
    severity = violence_section.find('span', class_=lambda x: x and x.startswith('ipl-status-pill'))

    return severity.text.strip() if severity else "N/A"


def scrape_imdb_data(imdb_ids):
    # List to store the scraped data
    scraped_data = []

    # Base URL for IMDb
    base_url = "https://www.imdb.com/title/"

    # Header to bypass IMDb's scraper block
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    }

    for i in range(0, len(imdb_ids), 2):  # Gets 1 of every 2 entries because every element is doubled
        # Create the URL for the IMDb page
        url = base_url + imdb_ids[i]

        # Send an HTTP GET request with headers to fetch the HTML content of the page
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to retrieve {imdb_ids[i]}")
            continue

        #Ensure the encoding is UTF-8
        response.encoding = 'utf-8'

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract relevant data (example fields)
        try:
            # Get the title of the series
            title = soup.find('h1').text.strip()


            # Get the rating of the series (<span class="sc-eb51e184-1 ljxVSS">6.5</span>)
            rating_element = soup.find('span', {'class': 'sc-eb51e184-1 ljxVSS'})
            rating = float(rating_element.text.strip()) if rating_element else "N/A"


            #Get the amount of user ratings (<div class="sc-eb51e184-3 kgbSIj">120</div>)
            #rating_count = soup.find('span' , {'class': 'sc-eb51e184-3 kgbSIj'}).text.strip() if soup.find('span' , {'class': 'sc-eb51e184-3 kgbSIj'}) else "0"

            # Get the amount of user ratings (<div class="sc-eb51e184-3 kgbSIj">120</div>)
            rating_count = soup.find('div', {'class': 'sc-eb51e184-3 kgbSIj'}).text.strip() if soup.find('div', {'class': 'sc-eb51e184-3 kgbSIj'}) else "0"

            # Get the start date (if available)
            release_date = soup.find('a', {'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}, href=lambda href: href and '/releaseinfo' in href)
            release_date_text = release_date.text.strip() if release_date else "N/A"

            # Get the number of episodes (if it's a series) (<span class="ipc-title__subtext">44</span>)
            #episodes = soup.find('span', {'class': 'ipc-title__subtext'}).text.strip() if soup.find('span', {'class': 'ipc-title__subtext'}) else "N/A"

            # Get the number of episodes (if it's a series)
            episodes_element = soup.find('h3', string=lambda text: text and "Episodes" in text)
            episodes_text = episodes_element.find_next('span', class_='ipc-title__subtext').text.strip() if episodes_element else "N/A"
            try:
                episodes = int(episodes_text) if episodes_text.isdigit() else "N/A"
            except ValueError:
                episodes = "N/A"


            # Get the parental certificate or guide
            violence_rating = get_parents_guide_violence(imdb_ids[i], headers)

            # Get the stars (actors) listed
            stars_section = soup.find_all('a', href=lambda href: href and "/name/" in href)
            stars = ', '.join([star.text.strip() for star in stars_section])  # (Not) limiting to the first 5 stars

            # Get the production company info (if available)
            #production_company = soup.find('a', {'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'})
            #production_company_name= production_company.text.strip() if production_company else "N/A"
            production_company = soup.find('a', {'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}, href=lambda href: href and '/company/' in href)
            production_company_name = production_company.text.strip() if production_company else "N/A"



            # Add the scraped data to the list
            scraped_data.append([imdb_ids[i], title, rating, rating_count, release_date_text, episodes, violence_rating, stars, production_company_name])

        except Exception as e:
            print(f"Error scraping {imdb_ids[i]}: {e}")

    return scraped_data

def write_to_csv(scraped_data, output_file):
    # Define the headers for the CSV file
    headers = ["IMDB_ID", "Title", "Rating", "Rating_Count", "Release_Date", "Episode_Count", "Violence", "Stars", "Production"]

    # Write the data to a CSV file
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        # Write headers as a row
        writer.writerow(headers)
        # Write each entry in scraped_data as a separate row
        for row in scraped_data:
            # Ensure that ratings are formatted as floats with one decimal place
            row[2] = f"{float(row[2]):.1f}" if row[2] != "N/A" else row[2]
            writer.writerow(row)

        """
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerows(scraped_data)
        """




# Example usage
scraped_data = scrape_imdb_data(imdb_ids)
write_to_csv(scraped_data, 'imdb_scraped_data_final.csv')


# **Check which files are present in the directory**

In [ ]:
import os
print(os.listdir())


['.config', '.ipynb_checkpoints', 'imdb_scraped_data.csv', 'fullhtmltext_filtered.txt', 'samplehtmltext.txt', 'sample_data']


In [ ]:
from google.colab import files
files.download('imdb_scraped_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>